# Module 3: Filtering & Enhancement

[![Open In Colab](https://colab.research.google.com/assets/colab-badge.svg)](https://colab.research.google.com/github/adiel2012/computer-graphics/blob/main/notebooks/03_Filtering_Enhancement.ipynb)

**Week 5-6: Image Filtering & Enhancement Techniques**

In [ ]:
import cv2
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline

## Topics
- Convolution fundamentals
- Gaussian Blur
- Median Filter
- Bilateral Filter
- Image Sharpening

**Key Functions**: `cv2.filter2D()`, `cv2.GaussianBlur()`, `cv2.medianBlur()`, `cv2.bilateralFilter()`

## 3.1 Understanding Convolution

**Convolution** is the fundamental operation in image filtering. It involves sliding a small matrix (kernel/filter) over the image and computing weighted sums.

**Mathematical Definition**:
```
G[i,j] = sum over k,l of: F[i+k, j+l] * K[k,l]
```

Where:
- `F` is the input image
- `K` is the kernel
- `G` is the output image

In [ ]:
# Download sample image
!wget -q https://upload.wikimedia.org/wikipedia/commons/thumb/3/3a/Cat03.jpg/481px-Cat03.jpg -O cat.jpg
img = cv2.imread('cat.jpg')
img_rgb = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)
gray = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)

print(f"Image shape: {img.shape}")
plt.figure(figsize=(8, 6))
plt.imshow(img_rgb)
plt.title('Original Image')
plt.axis('off')
plt.show()

### Simple Box Blur Kernel (Averaging)

In [ ]:
# Create a simple 5x5 averaging kernel
kernel_size = 5
kernel = np.ones((kernel_size, kernel_size), np.float32) / (kernel_size ** 2)

print("Box blur kernel (5x5):")
print(kernel)
print(f"\nSum of kernel: {kernel.sum():.2f} (should be 1.0 for brightness preservation)")

# Apply convolution
blurred = cv2.filter2D(gray, -1, kernel)

# Display
fig, axes = plt.subplots(1, 2, figsize=(12, 6))
axes[0].imshow(gray, cmap='gray')
axes[0].set_title('Original Grayscale')
axes[0].axis('off')

axes[1].imshow(blurred, cmap='gray')
axes[1].set_title('Box Blur (5x5)')
axes[1].axis('off')

plt.tight_layout()
plt.show()

### Custom Kernels - Edge Detection

In [ ]:
# Vertical edge detection kernel
kernel_vertical = np.array([[-1, 0, 1],
                            [-2, 0, 2],
                            [-1, 0, 1]], dtype=np.float32)

# Horizontal edge detection kernel
kernel_horizontal = np.array([[-1, -2, -1],
                               [ 0,  0,  0],
                               [ 1,  2,  1]], dtype=np.float32)

# Sharpening kernel
kernel_sharpen = np.array([[ 0, -1,  0],
                           [-1,  5, -1],
                           [ 0, -1,  0]], dtype=np.float32)

# Apply kernels
edges_v = cv2.filter2D(gray, -1, kernel_vertical)
edges_h = cv2.filter2D(gray, -1, kernel_horizontal)
sharpened = cv2.filter2D(gray, -1, kernel_sharpen)

# Display results
fig, axes = plt.subplots(2, 2, figsize=(12, 12))

axes[0, 0].imshow(gray, cmap='gray')
axes[0, 0].set_title('Original')

axes[0, 1].imshow(edges_v, cmap='gray')
axes[0, 1].set_title('Vertical Edges')

axes[1, 0].imshow(edges_h, cmap='gray')
axes[1, 0].set_title('Horizontal Edges')

axes[1, 1].imshow(sharpened, cmap='gray')
axes[1, 1].set_title('Sharpened')

for ax in axes.flat:
    ax.axis('off')

plt.tight_layout()
plt.show()

## 3.2 Gaussian Blur - Smart Smoothing

Gaussian blur uses a weighted average where nearby pixels have more influence. This creates a natural-looking blur.

**Formula**: The kernel follows a Gaussian (bell curve) distribution:
```
G(x, y) = (1/(2*pi*sigma^2)) * exp(-(x^2 + y^2)/(2*sigma^2))
```

**Parameters**:
- Kernel size: Must be odd (3x3, 5x5, 7x7, etc.)
- Sigma (σ): Controls blur strength (larger = more blur)

In [ ]:
# Apply Gaussian blur with different kernel sizes
gaussian_3 = cv2.GaussianBlur(gray, (3, 3), 0)
gaussian_7 = cv2.GaussianBlur(gray, (7, 7), 0)
gaussian_15 = cv2.GaussianBlur(gray, (15, 15), 0)
gaussian_31 = cv2.GaussianBlur(gray, (31, 31), 0)

# Display comparison
fig, axes = plt.subplots(2, 3, figsize=(15, 10))

axes[0, 0].imshow(gray, cmap='gray')
axes[0, 0].set_title('Original')

axes[0, 1].imshow(gaussian_3, cmap='gray')
axes[0, 1].set_title('Gaussian 3x3')

axes[0, 2].imshow(gaussian_7, cmap='gray')
axes[0, 2].set_title('Gaussian 7x7')

axes[1, 0].imshow(gaussian_15, cmap='gray')
axes[1, 0].set_title('Gaussian 15x15')

axes[1, 1].imshow(gaussian_31, cmap='gray')
axes[1, 1].set_title('Gaussian 31x31')

axes[1, 2].axis('off')

for ax in axes.flat[:-1]:
    ax.axis('off')

plt.tight_layout()
plt.show()

print("Notice: Larger kernel = stronger blur")

### Different Sigma Values

In [ ]:
# Fixed kernel size, varying sigma
kernel_size = (21, 21)

gaussian_sigma1 = cv2.GaussianBlur(gray, kernel_size, 1)
gaussian_sigma5 = cv2.GaussianBlur(gray, kernel_size, 5)
gaussian_sigma10 = cv2.GaussianBlur(gray, kernel_size, 10)

fig, axes = plt.subplots(1, 4, figsize=(16, 4))

axes[0].imshow(gray, cmap='gray')
axes[0].set_title('Original')

axes[1].imshow(gaussian_sigma1, cmap='gray')
axes[1].set_title('Sigma = 1')

axes[2].imshow(gaussian_sigma5, cmap='gray')
axes[2].set_title('Sigma = 5')

axes[3].imshow(gaussian_sigma10, cmap='gray')
axes[3].set_title('Sigma = 10')

for ax in axes:
    ax.axis('off')

plt.tight_layout()
plt.show()

## 3.3 Median Filter - Noise Removal Champion

**How it works**: Replace each pixel with the **median** value in its neighborhood.

**Best for**: Salt-and-pepper noise (random black/white pixels)

**Advantage**: Preserves edges better than Gaussian blur!

In [ ]:
# Add salt-and-pepper noise
noisy = gray.copy()
noise_ratio = 0.02

# Salt (white pixels)
num_salt = int(noise_ratio * gray.size)
coords = [np.random.randint(0, i, num_salt) for i in gray.shape]
noisy[coords[0], coords[1]] = 255

# Pepper (black pixels)
num_pepper = int(noise_ratio * gray.size)
coords = [np.random.randint(0, i, num_pepper) for i in gray.shape]
noisy[coords[0], coords[1]] = 0

# Apply different filters
gaussian_denoised = cv2.GaussianBlur(noisy, (5, 5), 0)
median_denoised = cv2.medianBlur(noisy, 5)

# Display comparison
fig, axes = plt.subplots(2, 2, figsize=(12, 12))

axes[0, 0].imshow(gray, cmap='gray')
axes[0, 0].set_title('Original')

axes[0, 1].imshow(noisy, cmap='gray')
axes[0, 1].set_title('Noisy (Salt & Pepper)')

axes[1, 0].imshow(gaussian_denoised, cmap='gray')
axes[1, 0].set_title('Gaussian Blur Denoised')

axes[1, 1].imshow(median_denoised, cmap='gray')
axes[1, 1].set_title('Median Filter Denoised (Better!)')

for ax in axes.flat:
    ax.axis('off')

plt.tight_layout()
plt.show()

print("Notice: Median filter removes noise while preserving edges better!")

## 3.4 Bilateral Filter - Edge-Preserving Blur

**The Problem**: Gaussian blur smooths everything, including edges!

**The Solution**: Bilateral filter considers **both**:
1. Spatial distance (like Gaussian)
2. Intensity difference (preserves edges)

**Result**: Smooth regions are blurred, but edges stay sharp!

**Parameters**:
- `d`: Diameter of pixel neighborhood
- `sigmaColor`: Color space sigma (larger = more colors mixed)
- `sigmaSpace`: Coordinate space sigma (larger = farther pixels influence)

In [ ]:
# Apply bilateral filter with different parameters
bilateral_9_75_75 = cv2.bilateralFilter(gray, 9, 75, 75)
bilateral_15_80_80 = cv2.bilateralFilter(gray, 15, 80, 80)

# Compare with Gaussian
gaussian_comp = cv2.GaussianBlur(gray, (15, 15), 0)

# Display comparison
fig, axes = plt.subplots(2, 2, figsize=(12, 12))

axes[0, 0].imshow(gray, cmap='gray')
axes[0, 0].set_title('Original')

axes[0, 1].imshow(gaussian_comp, cmap='gray')
axes[0, 1].set_title('Gaussian Blur (edges blurred)')

axes[1, 0].imshow(bilateral_9_75_75, cmap='gray')
axes[1, 0].set_title('Bilateral Filter (d=9)\n(edges preserved!)')

axes[1, 1].imshow(bilateral_15_80_80, cmap='gray')
axes[1, 1].set_title('Bilateral Filter (d=15)\n(stronger smoothing)')

for ax in axes.flat:
    ax.axis('off')

plt.tight_layout()
plt.show()

print("Use bilateral filter for:")
print("- Portrait smoothing (skin texture)")
print("- Cartoonization effects")
print("- Edge-aware denoising")

## 3.5 Image Sharpening

**Concept**: Enhance edges to make images appear sharper.

**Method 1**: Unsharp Masking
```
sharpened = original + (original - blurred) * amount
```

**Method 2**: Sharpening kernel (convolution)

In [ ]:
# Method 1: Unsharp Masking
blurred = cv2.GaussianBlur(gray, (9, 9), 10)
unsharp_mask = cv2.addWeighted(gray, 1.5, blurred, -0.5, 0)

# Method 2: Sharpening Kernel
kernel_sharpen = np.array([[ 0, -1,  0],
                           [-1,  5, -1],
                           [ 0, -1,  0]])
kernel_sharpened = cv2.filter2D(gray, -1, kernel_sharpen)

# Strong sharpening kernel
kernel_sharpen_strong = np.array([[-1, -1, -1],
                                  [-1,  9, -1],
                                  [-1, -1, -1]])
strong_sharpened = cv2.filter2D(gray, -1, kernel_sharpen_strong)

# Display
fig, axes = plt.subplots(2, 2, figsize=(12, 12))

axes[0, 0].imshow(gray, cmap='gray')
axes[0, 0].set_title('Original')

axes[0, 1].imshow(unsharp_mask, cmap='gray')
axes[0, 1].set_title('Unsharp Masking')

axes[1, 0].imshow(kernel_sharpened, cmap='gray')
axes[1, 0].set_title('Kernel Sharpening (Mild)')

axes[1, 1].imshow(strong_sharpened, cmap='gray')
axes[1, 1].set_title('Kernel Sharpening (Strong)')

for ax in axes.flat:
    ax.axis('off')

plt.tight_layout()
plt.show()

print("Warning: Over-sharpening can create artifacts!")

## 3.6 Filter Comparison - Which to Use When?

In [ ]:
# Side-by-side comparison on color image
img_blur_gaussian = cv2.GaussianBlur(img, (15, 15), 0)
img_blur_median = cv2.medianBlur(img, 15)
img_blur_bilateral = cv2.bilateralFilter(img, 15, 80, 80)

fig, axes = plt.subplots(2, 2, figsize=(14, 14))

axes[0, 0].imshow(cv2.cvtColor(img, cv2.COLOR_BGR2RGB))
axes[0, 0].set_title('Original', fontsize=14)

axes[0, 1].imshow(cv2.cvtColor(img_blur_gaussian, cv2.COLOR_BGR2RGB))
axes[0, 1].set_title('Gaussian Blur\nFast, smooth everything', fontsize=14)

axes[1, 0].imshow(cv2.cvtColor(img_blur_median, cv2.COLOR_BGR2RGB))
axes[1, 0].set_title('Median Filter\nBest for salt-pepper noise', fontsize=14)

axes[1, 1].imshow(cv2.cvtColor(img_blur_bilateral, cv2.COLOR_BGR2RGB))
axes[1, 1].set_title('Bilateral Filter\nEdge-preserving, slower', fontsize=14)

for ax in axes.flat:
    ax.axis('off')

plt.tight_layout()
plt.show()

print("Filter Selection Guide:")
print("=" * 60)
print("Gaussian Blur: General smoothing, fast")
print("Median Filter: Salt-and-pepper noise removal")
print("Bilateral Filter: Smoothing while preserving edges")
print("Box Blur: Fastest, but lowest quality")

## Project: Build a Photo Filter App

In [ ]:
def apply_vintage_filter(image):
    """Apply vintage/retro filter"""
    # Slight blur for dreamy effect
    img = cv2.GaussianBlur(image, (5, 5), 0)
    
    # Warm color tone (boost red/yellow channels)
    img = img.astype(np.float32)
    img[:, :, 0] *= 0.8  # Reduce blue
    img[:, :, 1] *= 1.1  # Boost green slightly
    img[:, :, 2] *= 1.2  # Boost red
    img = np.clip(img, 0, 255).astype(np.uint8)
    
    return img

def apply_sharp_filter(image):
    """Apply sharpening filter"""
    kernel = np.array([[-1, -1, -1],
                       [-1,  9, -1],
                       [-1, -1, -1]])
    return cv2.filter2D(image, -1, kernel)

def apply_cartoon_filter(image):
    """Apply cartoonization effect"""
    # Edge-preserving smoothing
    smooth = cv2.bilateralFilter(image, 9, 75, 75)
    
    # Apply bilateral again for stronger effect
    smooth = cv2.bilateralFilter(smooth, 9, 75, 75)
    
    return smooth

# Apply all filters
vintage = apply_vintage_filter(img)
sharp = apply_sharp_filter(img)
cartoon = apply_cartoon_filter(img)

# Display
fig, axes = plt.subplots(2, 2, figsize=(14, 14))

axes[0, 0].imshow(cv2.cvtColor(img, cv2.COLOR_BGR2RGB))
axes[0, 0].set_title('Original', fontsize=14)

axes[0, 1].imshow(cv2.cvtColor(vintage, cv2.COLOR_BGR2RGB))
axes[0, 1].set_title('Vintage Filter', fontsize=14)

axes[1, 0].imshow(cv2.cvtColor(sharp, cv2.COLOR_BGR2RGB))
axes[1, 0].set_title('Sharp Filter', fontsize=14)

axes[1, 1].imshow(cv2.cvtColor(cartoon, cv2.COLOR_BGR2RGB))
axes[1, 1].set_title('Cartoon Filter', fontsize=14)

for ax in axes.flat:
    ax.axis('off')

plt.tight_layout()
plt.show()

# Save filtered images
cv2.imwrite('vintage.jpg', vintage)
cv2.imwrite('sharp.jpg', sharp)
cv2.imwrite('cartoon.jpg', cartoon)
print("Filtered images saved!")

## Summary

You learned:
- **Convolution**: The fundamental operation for image filtering
- **Gaussian Blur**: Natural-looking smoothing with bell curve weights
- **Median Filter**: Best for removing salt-and-pepper noise
- **Bilateral Filter**: Edge-preserving smoothing (cartoonization, portraits)
- **Image Sharpening**: Enhance edges using unsharp masking or kernels

**Key Insights**:
1. Convolution = sliding a kernel over the image
2. Kernel weights determine the effect (blur, sharpen, detect edges)
3. Gaussian preserves brightness (kernel sums to 1)
4. Median is non-linear (no convolution!)
5. Bilateral considers both space AND intensity

**Next**: [Module 4 - Geometric Transformations](04_Module.ipynb)